# Notebook to create a new LUX-SAF Wikibase instance
This script builds on an excel file that is provided by the data modellers. That excel file **TODO** Provide a link to repository containing the latest version(s). 
That file contains the CIDOC-CRM representation of the SAF with mappings to the perceived Wikibase(Qualifier). 

In [51]:
from wikidataintegrator import wdi_core, wdi_login, wdi_config
from getpass import getpass
import pandas as pd

wbstack = "lux-saf-beta"
wikibase = "https://{}.wiki.opencura.com/".format(wbstack)
api = "https://{}.wiki.opencura.com/w/api.php".format(wbstack)
sparql = "https://{}.wiki.opencura.com/query/sparql".format(wbstack)
entityUri = wikibase.replace("https:", "http:")+"entity/"

WBUSER = "Andrawaag"
WBPASS = getpass()
login = wdi_login.WDLogin(WBUSER, WBPASS, mediawiki_api_url=api)
localEntityEngine = wdi_core.WDItemEngine.wikibase_item_engine_factory(api,sparql)

········


# import data
#### TODO
* read directly from the versioned repository/Gdrive

In [52]:
model_def = pd.read_excel("../DM_SAF/DM_SAF_vers.1.0.3_andra.xlsx", header=1)
model_def

,Data type,"Description - see document ""Data model - minimum set of fields: definitions""",Deutsch,English,français
0,string,free text,Name,name,nom
1,wikibase-item,closed controlled list (CL3),Namensformat,name format,format du nom
2,string,free text,Numerierung,numeration,numérotation
3,string,free text,Titel,title,titre
4,string,NaN,Quelle Text,source of information - text,source d'information - texte
5,wikibase-item,closed controlled list (CL6),Quellentyp,type of source,type de source d'information
6,wikibase-item,NaN,übergeordneter,superclass of,class supérieure du
7,wikibase-item,NaN,Untergeordneter,subclass of,class inférieure du
8,url,URL,Quelle URL,source of information - url,source d'information URL
9,string,free text,Alternativname,alternative name,nom alternatif


# Create properties

In [53]:
def createProperty(login=login, wdprop=None, lulabel="", enlabel="", frlabel="", delabel="", description="", property_datatype=""):
    if wdprop== None:
        s = []
    else:
        s = [wdi_core.WDUrl(wdprop, prop_nr="P1")]
    localEntityEngine = wdi_core.WDItemEngine.wikibase_item_engine_factory(api,sparql)
    item = localEntityEngine(data=s)
    if lulabel != "":
        item.set_label(lulabel, lang="lb")
    item.set_label(enlabel, lang="en")
    item.set_label(delabel, lang="de")
    item.set_label(frlabel, lang="fr")
    item.set_description(description, lang="en")
    
    print(item.write(login, entity_type="property", property_datatype=property_datatype))

# OWL properties to capture CIDOC-CRM

In [54]:
# instance of
createProperty(login, lulabel="ass eng", 
                      enlabel="instance of",
                      frlabel="instance de",
                      delabel="ist ein(e)",
                      property_datatype="wikibase-item")

# subclass of 
## 
createProperty(login, lulabel="Ënnerklass vu(n)", 
                      enlabel="subclass of",
                      frlabel="sous-classe de",
                      delabel="Unterklasse von",
                      property_datatype="wikibase-item")
# skos:exact match
createProperty(login, lulabel="genauen Match", 
                      enlabel="exact match",
                      frlabel="correspondance exacte",
                      delabel="exakte Übereinstimmung",
                      description="mapping",
                      property_datatype="url")
#domain 
createProperty(login, lulabel="domain", 
                      enlabel="domain",
                      frlabel="domaine",
                      delabel="domain",
                      property_datatype="wikibase-item")
#range
createProperty(login, lulabel="reechwäit", 
                      enlabel="range",
                      frlabel="intervalle",
                      delabel="reichweite",
                      property_datatype="wikibase-item")
#subPropertyOf
createProperty(login, lulabel="Ënnerbesëtz vun", 
                      enlabel="subproperty of",
                      frlabel="sous-propriété de",
                      delabel="untereigenschaft von",
                      property_datatype="wikibase-item")
#inverseOf
createProperty(login, lulabel="invers vun", 
                      enlabel="inverse of",
                      frlabel="inverse de",
                      delabel="invers von",
                      property_datatype="wikibase-item")
propertyID = dict()
query = """SELECT ?prop ?propLabel WHERE {
  ?prop wikibase:directClaim ?wdt .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}"""
for index, row in wdi_core.WDItemEngine.execute_sparql_query(query, as_dataframe = True, endpoint=sparql).iterrows():
    print(row["prop"].replace(entityUri, ""), row["propLabel"])
    propertyID[row["propLabel"]] = row["prop"].replace(entityUri, "")

# class item
localEntityEngine = wdi_core.WDItemEngine.wikibase_item_engine_factory(api,sparql)
item = localEntityEngine(new_item=True)
item.set_label("Class", lang="en")
item.set_aliases(["Owl:Class"], lang="en")
item.write(login)

# property item
localEntityEngine = wdi_core.WDItemEngine.wikibase_item_engine_factory(api,sparql)
item = localEntityEngine(new_item=True)
item.set_label("Property", lang="en")
item.set_aliases(["owl:ObjectProperty"], lang="en")
item.write(login)

/usr/local/lib/python3.9/site-packages/wikidataintegrator/wdi_core.py:485: UserWarning: Warning: No distinct value properties found
Please set P2302 and Q21502410 in your wikibase or set `core_props` manually.
Continuing with no core_props
  warnings.warn("Warning: No distinct value properties found\n" +


P1
P2
P3
P4
P5
P6
P7
P1 instance of
P2 subclass of
P3 exact match
P4 domain
P5 range
P6 subproperty of
P7 inverse of


'Q2'

# Read the property definitions from the DM_SAF 

In [55]:
for index, row in model_def.iterrows():
    if row["Data type"].strip() in wdi_config.property_value_types.keys():
        print(row["Data type"])
        try:
            createProperty(login, enlabel=row["English"], frlabel=row["français"], delabel=row["Deutsch"], description="Lux SAF Property", property_datatype=row["Data type"].strip()) 
        except:
            print("Error with ", row["English"])
    else:
        print("Error", row["Data type"])

string
P8
wikibase-item
P9
string
P10
string
P11
string
P12
wikibase-item
P13
wikibase-item
P14
wikibase-item
Error while writing to Wikidata
Error with  subclass of
url
P16
string
P17
edtf
P18
string
P19
edtf
P20
string
P21
wikibase-item
P22
string
P23
edtf
P24
edtf
P25
string
P26
string
P27
string
P28
external-id
P29
string
P30
string
P31
edtf
P32
edtf
P33
string
P34
string
P35
external-id
P36
external-id
P37
external-id
P38
url
P39
url
P40


# Create the controlled lists as described in DM_SAF
## CL4 Gender

In [56]:
CL4 = pd.read_excel("../DM_SAF/DM_SAF_vers.1.0.2_andra.xlsx", sheet_name="CL4 GENDER")
for index, row in CL4.iterrows():
    print(row["Label (English)"])
    item = localEntityEngine(new_item=True)
    item.set_label(row["Label (English)"], lang="en")
    item.set_label(row["Label (German)"], lang="de")
    item.set_label(row["Label (French)"], lang="fr")
    print(item.write(login))

male
Q3
female
Q4
not known
Q5
not applicable
Q6


## CL5 STATUS

In [57]:
CL5 = pd.read_excel("../DM_SAF/DM_SAF_vers.1.0.2_andra.xlsx", sheet_name="CL5 STATUS")
for index, row in CL5.iterrows():
    print(row["Label (English)"])
    item = localEntityEngine(new_item=True)
    item.set_label(row["Label (English)"], lang="en")
    item.set_label(row["Label (German)"], lang="de")
    item.set_label(row["Label (French)"], lang="fr")
    print(item.write(login))

Draft
Q7
To be validated
Q8
To be clarified
Q9
Validated
Q10
To be published
Q11
Published
Q12
To be updated
Q13
Updated
Q14


In [58]:
CL3 = pd.read_excel("../DM_SAF/DM_SAF_vers.1.0.2_andra.xlsx", sheet_name="CL3 Name Format")
for index, row in CL3.iterrows():
    item = localEntityEngine(new_item=True)
    item.set_label(row["Cataloging specs"])
    print(item.write(login))

Q15
Q16


# manually added external identifiers not yet covered in DMG

In [59]:
#ARK
createProperty(login, lulabel="ARK", 
                      enlabel="ARK",
                      frlabel="ARK",
                      delabel="ARK",
                      property_datatype="url")


P41


In [1]:
person_item = localEntityEngine(new_item=True)
person_item.set_label("E21 Person", lang="en")
print(person_item.write(login))